In [46]:
import json
import os
import pprint
from data_adaptor import DataAdaptor

In [47]:
N_EXAMPLES = 500

# 2WikiMultihopQA
Authors only use question-answer information. The context is not provided.
> "we use only the question-answer pairs from these datasets, not any passages of relevant text that they contain. These datasets both contain 2-hop compositional questions sourced from facts that appear in Wikipedia articles." - Press, et al.

Authors do not use this dataset to measure compositionality gap which requires known sub-questions and answers to measure.
> "Note that the rest of this section shows that elicitive prompts improve performance but does not show that they narrow the compositionality gap since we lack sub-questions for datasets other than CC." - Press, et al.

In [48]:
from data_loaders import load_2WikiMultihopQA

In [49]:
# Load the training data
wiki_sample = load_2WikiMultihopQA(n_examples=N_EXAMPLES, split='train')

In [50]:
# Print the first example in pretty format
print(json.dumps(wiki_sample[0], indent=4))

{
    "_id": "13f5ad2c088c11ebbd6fac1f6bf848b6",
    "type": "bridge_comparison",
    "question": "Are director of film Move (1970 Film) and director of film M\u00e9diterran\u00e9e (1963 Film) from the same country?",
    "context": [
        [
            "Stuart Rosenberg",
            [
                "Stuart Rosenberg (August 11, 1927 \u2013 March 15, 2007) was an American film and television director whose motion pictures include \"Cool Hand Luke\" (1967), \"Voyage of the Damned\" (1976), \"The Amityville Horror\" (1979), and \"The Pope of Greenwich Village\" (1984).",
                "He was noted for his work with actor Paul Newman."
            ]
        ],
        [
            "M\u00e9diterran\u00e9e (1963 film)",
            [
                "M\u00e9diterran\u00e9e is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schl\u00f6ndorff.",
                "It was written by Philippe Sollers and produced by Barbet Schroeder, with music 

Let's look at the evidence required to answer the question. It's possible we create prompt examples using these evidences to insert into the fine-tuning set. For example,

**Examplar:**
```
Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg.
Follow up: Who is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no
```

In [51]:
wiki_sample[0]['supporting_facts']

[['Move (1970 film)', 0],
 ['Méditerranée (1963 film)', 0],
 ['Stuart Rosenberg', 0],
 ['Jean-Daniel Pollet', 0]]

### Adapting to Self-Ask Examplar

In [52]:
wiki_adaptor = DataAdaptor(dataset="2WikiMultihopQA")

In [53]:
# wiki_examplars = wiki_adaptor.generate_examplars(wiki_sample, strategy="self-ask")
for examplar in wiki_examplars:
    print(examplar)

Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no

Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of The Falcon (film)?
Intermediate answer: Vatroslav Mimica
Follow up: Who is the director of Valentin the Good?
Intermediate answer: Martin Frič
Follow up: What is the country of citizenship of Vatroslav Mimica?
Intermediate answer: Croatian
Follow

### Adapting to Self-Ask Training Example
We can augment the target texts in the dataset with the self-ask rationale to fine-tune a language model to generate text with the self-ask rationale.

In [54]:
wiki_training_examples = wiki_adaptor.generate_training_examples(wiki_sample, strategy="self-ask")
for training_example in wiki_training_examples:
    print(json.dumps(training_example, indent=4))

Generating 2WikiMultihopQA self-ask training examples: 100%|███████████████████████████████| 500/500 [00:03<00:00, 153.93it/s]
Structuring 2WikiMultihopQA self-ask training examples: 100%|█████████████████████████████| 500/500 [00:00<00:00, 2633.74it/s]

{
    "prompt": "Fact #0: Move is a 1970 American comedy film starring Elliott Gould, Paula Prentiss and Genevi\u00e8ve Wa\u00efte, and directed by Stuart Rosenberg.\nFact #1: M\u00e9diterran\u00e9e is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schl\u00f6ndorff.\nFact #2: Stuart Rosenberg (August 11, 1927 \u2013 March 15, 2007) was an American film and television director whose motion pictures include \"Cool Hand Luke\" (1967), \"Voyage of the Damned\" (1976), \"The Amityville Horror\" (1979), and \"The Pope of Greenwich Village\" (1984).\nFact #3: Jean-Daniel Pollet (1936\u20132004) was a French film director and screenwriter who was most active in the 1960s and 1970s.\n\nQuestion: Are director of film Move (1970 Film) and director of film M\u00e9diterran\u00e9e (1963 Film) from the same country?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: Who is the director of Move (1970 film)?\nIntermediate answer: Stuart

In [55]:
print(wiki_training_examples[0]["prompt"])
print(wiki_training_examples[0]["target"])

Fact #0: Move is a 1970 American comedy film starring Elliott Gould, Paula Prentiss and Geneviève Waïte, and directed by Stuart Rosenberg.
Fact #1: Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff.
Fact #2: Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984).
Fact #3: Jean-Daniel Pollet (1936–2004) was a French film director and screenwriter who was most active in the 1960s and 1970s.

Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here:

Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jea

### Direct Prompting Training Examples
Simply provide the facts and ask the question. No thought variable or rationale involved.

In [67]:
direct_training_examples = wiki_adaptor.generate_training_examples(
    wiki_sample,
    strategy="direct"
)
len(direct_training_examples)

Generating 2WikiMultihopQA direct training examples: 100%|███████████████████████████████| 500/500 [00:00<00:00, 27090.44it/s]
Structuring 2WikiMultihopQA direct training examples: 100%|███████████████████████████████| 500/500 [00:00<00:00, 3823.19it/s]


500

In [68]:
print("--------- Augmented Prompt ---------")
print(direct_training_examples[0]["prompt"])
print("--------- Target ---------")
print(direct_training_examples[0]["target"])

--------- Augmented Prompt ---------
Fact #0: Move is a 1970 American comedy film starring Elliott Gould, Paula Prentiss and Geneviève Waïte, and directed by Stuart Rosenberg.
Fact #1: Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff.
Fact #2: Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984).
Fact #3: Jean-Daniel Pollet (1936–2004) was a French film director and screenwriter who was most active in the 1960s and 1970s.

Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Answer:
--------- Target ---------
no


### Augment with In-Context Examplars and Self-Ask Rationale Targets
We can combine the two above to create an augmented fine-tuning dataset:
1. Prompt text has in-context examplars
2. Target text has the self-ask rationale

In [59]:
training_examplars = wiki_examplars[:4]
augmented_example = wiki_adaptor.generate_training_examples(
    wiki_sample[0], 
    strategy="self-ask", 
    examplars=training_examplars
    )[0]
print("--------- Augmented Prompt ---------")
print(augmented_example["prompt"])
print("--------- Target ---------")
print(augmented_example["target"])

Generating 2WikiMultihopQA self-ask training examples: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 81.81it/s]
Structuring 2WikiMultihopQA self-ask training examples: 100%|██████████████████████████████████| 1/1 [00:00<00:00, 575.90it/s]

--------- Augmented Prompt ---------
Example Response
Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no

Example Response
Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of The Falcon (film)?
Intermediate answer: Vatroslav Mimica
Follow up: Who is the director of Valentin the Good?
Intermediate answer: Martin Frič
Follow up: What is the country o

In [60]:
training_examplars = wiki_examplars[:2]
augmented_examples = wiki_adaptor.generate_training_examples(
    wiki_sample, 
    strategy="self-ask",
    examplars=training_examplars
    )

# look at token counts in prompt (context size)
print("context size")
for example in augmented_examples:
    print(example["num_prompt_tokens"])

# look at token counts
print("total tokens")
for example in augmented_examples:
    print(example["num_tokens"])

Generating 2WikiMultihopQA self-ask training examples: 100%|███████████████████████████████| 500/500 [00:03<00:00, 160.70it/s]
Structuring 2WikiMultihopQA self-ask training examples: 100%|█████████████████████████████| 500/500 [00:00<00:00, 1235.83it/s]

context size
517
532
461
374
433
412
393
439
384
374
413
385
410
370
375
377
376
415
381
413
506
390
363
389
432
439
476
372
460
384
473
389
389
371
530
433
426
391
351
392
459
386
351
481
370
507
501
477
393
446
380
396
369
375
387
469
362
363
382
440
392
466
438
385
391
398
380
400
383
409
410
363
457
391
391
396
378
451
409
376
373
395
358
485
382
483
390
401
480
519
386
364
383
414
407
462
371
400
370
539
536
462
369
451
457
401
491
466
391
398
420
418
383
490
400
443
393
375
399
439
421
374
383
405
364
381
466
484
405
400
375
393
364
376
376
469
387
445
482
415
408
402
362
485
400
386
384
392
404
489
494
416
373
394
365
439
516
483
454
369
417
480
399
432
488
409
480
385
377
380
400
419
373
390
474
374
374
394
458
485
448
410
377
363
380
455
388
422
493
416
481
402
399
355
424
432
399
408
401
369
467
461
416
467
366
389
440
453
371
398
388
375
406
363
410
376
459
405
415
460
385
454
356
373
384
474
374
375
367
469
429
376
361
351
417
415
376
417
355
395
389
389
370
366
476
391
372

In [61]:
print(augmented_examples[1]["prompt"])

Example Response
Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no

Example Response
Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of The Falcon (film)?
Intermediate answer: Vatroslav Mimica
Follow up: Who is the director of Valentin the Good?
Intermediate answer: Martin Frič
Follow up: What is the country of citizenship of Vatroslav Mimica?
In

# Format for alpaca

In [69]:
alpaca_training_examples = []
for example in direct_training_examples:
    prompt_split = example['prompt'].split('\n\n')
    current_example = {'instruction': prompt_split[1],
        'input': prompt_split[0],
        'output': example['target']}
    alpaca_training_examples.append(current_example)

pprint.pprint(alpaca_training_examples)

[{'input': 'Fact #0: Move is a 1970 American comedy film starring Elliott '
           'Gould, Paula Prentiss and Geneviève Waïte, and directed by Stuart '
           'Rosenberg.\n'
           'Fact #1: Méditerranée is a 1963 French experimental film directed '
           'by Jean-Daniel Pollet with assistance from Volker Schlöndorff.\n'
           'Fact #2: Stuart Rosenberg (August 11, 1927 – March 15, 2007) was '
           'an American film and television director whose motion pictures '
           'include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), '
           '"The Amityville Horror" (1979), and "The Pope of Greenwich '
           'Village" (1984).\n'
           'Fact #3: Jean-Daniel Pollet (1936–2004) was a French film director '
           'and screenwriter who was most active in the 1960s and 1970s.',
  'instruction': 'Question: Are director of film Move (1970 Film) and director '
                 'of film Méditerranée (1963 Film) from the same country?\n'
       

In [70]:
os.getcwd()

'/Users/adamweinberger/MIDS/compositional-reasoning-finetuning'

In [71]:
with open('data/Alpaca-LoRa/sample_alpaca.json', 'w') as f:
    json.dump(alpaca_training_examples, f)

In [65]:
with open('data/Alpaca-LoRa/alpaca-bitcoin-sentiment-dataset.json', 'r') as f:
    alpaca_bitcoin_example = json.load(f)
print(len(alpaca_bitcoin_example))

1897


# Train Model

## Setup